In [1]:
import numpy as np

import pandas as pd
import emoji
import contractions
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk import pos_tag
from nltk.corpus import wordnet

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('dataset/hasil_crawling.csv')
df.head(10)

,date,username,tweet,keyword
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat https://t.co/csHISa3WwB,jelek
1,2022-06-28 08:48:41+00:00,imasshana,"@convomfs mau req foto ktp pke filter ig deh, ...",jelek
2,2022-06-28 08:48:35+00:00,junhuicy,Nyoba pake kamera analog dan hasilnya jelek 🥲 ...,jelek
3,2022-06-28 08:48:33+00:00,melicoww,@cutiebunny04 @convomf jelek kali emotmu itu🤢 ...,jelek
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek
5,2022-06-28 08:48:18+00:00,renjunkw,@RnjntrA @huanarenjun Kebetulan ara jelek,jelek
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta. seandainya bi...,jelek
7,2022-06-28 08:47:59+00:00,nosyacey,@chioollo emg udh jelek sih tpi itu jeleknya p...,jelek
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek
9,2022-06-28 08:47:52+00:00,j23cat,HALO PLS LEWAT TL DONG HPKU KECEMPLUNG DI WC I...,jelek


In [3]:
#hapus emoji
import re
def remove_emojis(tweet):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)

In [4]:
#cleansing
def cleansing(tweet): 
    #replace RT tag didalam tweet
    t1 = re.sub('RT\s','',  tweet)
    
    #replace username (@)
    t2 = re.sub('\B@\w+','', t1)
    
    
    #menghapus URL
    t3 = re.sub('(http|https):\/\/\S+','', t2)
    
    #menghilangkan hastag
    t4 = re.sub('#+', '', t3)
    
      #menghilanglan tanda baca berulang
    t5 = re.sub (r'[\?\.\!]+(?=[\?.\!])', '', t4)
    
    #menghilangkan kata berulang (ooo -> oo)
    t6 = re.sub(r'(.)\1+', r'\1\1', t5)
    
     #menghilangkan angka dan spesial karakter dan hanya mengambil alpabet
    t7 = re.sub(r'[^a-zA-Z0-9]', ' ', t6)
    
    t8 = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", t7)
   
     #huruf kapital jadi huruf kecil (case folding)
    t9 = t8.lower()
    
    return t9

In [5]:
for i, r in df.iterrows():
    y = cleansing(r['tweet'])
    df.loc[i, 'tweet'] = y

In [6]:
df.to_csv('dataset/data_cleaning.csv', index=False)
df.head(10)

,date,username,tweet,keyword
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat,jelek
1,2022-06-28 08:48:41+00:00,imasshana,mau req foto ktp pke filter ig deh muka gue ...,jelek
2,2022-06-28 08:48:35+00:00,junhuicy,nyoba pake kamera analog dan hasilnya jelek ...,jelek
3,2022-06-28 08:48:33+00:00,melicoww,jelek kali emotmu itu yh yank kmu juga teta...,jelek
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek
5,2022-06-28 08:48:18+00:00,renjunkw,kebetulan ara jelek,jelek
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta seandainya bi...,jelek
7,2022-06-28 08:47:59+00:00,nosyacey,emg udh jelek sih tpi itu jeleknya pake bange...,jelek
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek
9,2022-06-28 08:47:52+00:00,j23cat,halo pls lewat tl dong hpku kecemplung di wc i...,jelek


In [7]:
df.shape

(1000, 4)

NORMALISASI BAHASA

In [8]:
#normalisasi
key_norm = pd.read_csv('dataset/kamus_normalisasi.csv')

def text_normalize(text):
    text = ' '.join([key_norm[key_norm['slang'] == word]['formal'].values[0] if (key_norm["slang"] 
    == word).any() else word for word in text.split()])
    text = str.lower(text)
    return text
df['Normalisasi'] = df['tweet'].apply(text_normalize)
df.head(10)

,date,username,tweet,keyword,Normalisasi
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat,jelek,cel ketawanya jelek banget
1,2022-06-28 08:48:41+00:00,imasshana,mau req foto ktp pke filter ig deh muka gue ...,jelek,mau request foto ktp pakai filter ig deh muka ...
2,2022-06-28 08:48:35+00:00,junhuicy,nyoba pake kamera analog dan hasilnya jelek ...,jelek,nyoba pake kamera analog dan hasilnya jelek in...
3,2022-06-28 08:48:33+00:00,melicoww,jelek kali emotmu itu yh yank kmu juga teta...,jelek,jelek kali emotmu itu yah sayang kamu juga tet...
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek,sepertinya nickname aku kalau di tambah hala j...
5,2022-06-28 08:48:18+00:00,renjunkw,kebetulan ara jelek,jelek,kebetulan ara jelek
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta seandainya bi...,jelek,beruntung banget cinta itu buta seandainya bis...
7,2022-06-28 08:47:59+00:00,nosyacey,emg udh jelek sih tpi itu jeleknya pake bange...,jelek,emang udah jelek sih tapi itu jeleknya pake ba...
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek,aku sayang banget sama provider tri meskipun s...
9,2022-06-28 08:47:52+00:00,j23cat,halo pls lewat tl dong hpku kecemplung di wc i...,jelek,halo tolong lewat timeline dong hpku kecemplun...


TOKENIZING

In [9]:
def tokenization(tweet):
    tweet =re.split('\W+', tweet)
    return tweet
df['TOKENIZATION'] = df['Normalisasi'].apply(lambda x: tokenization(x.lower()))
df.head(10)

,date,username,tweet,keyword,Normalisasi,TOKENIZATION
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat,jelek,cel ketawanya jelek banget,"[cel, ketawanya, jelek, banget]"
1,2022-06-28 08:48:41+00:00,imasshana,mau req foto ktp pke filter ig deh muka gue ...,jelek,mau request foto ktp pakai filter ig deh muka ...,"[mau, request, foto, ktp, pakai, filter, ig, d..."
2,2022-06-28 08:48:35+00:00,junhuicy,nyoba pake kamera analog dan hasilnya jelek ...,jelek,nyoba pake kamera analog dan hasilnya jelek in...,"[nyoba, pake, kamera, analog, dan, hasilnya, j..."
3,2022-06-28 08:48:33+00:00,melicoww,jelek kali emotmu itu yh yank kmu juga teta...,jelek,jelek kali emotmu itu yah sayang kamu juga tet...,"[jelek, kali, emotmu, itu, yah, sayang, kamu, ..."
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek,sepertinya nickname aku kalau di tambah hala j...,"[sepertinya, nickname, aku, kalau, di, tambah,..."
5,2022-06-28 08:48:18+00:00,renjunkw,kebetulan ara jelek,jelek,kebetulan ara jelek,"[kebetulan, ara, jelek]"
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta seandainya bi...,jelek,beruntung banget cinta itu buta seandainya bis...,"[beruntung, banget, cinta, itu, buta, seandain..."
7,2022-06-28 08:47:59+00:00,nosyacey,emg udh jelek sih tpi itu jeleknya pake bange...,jelek,emang udah jelek sih tapi itu jeleknya pake ba...,"[emang, udah, jelek, sih, tapi, itu, jeleknya,..."
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek,aku sayang banget sama provider tri meskipun s...,"[aku, sayang, banget, sama, provider, tri, mes..."
9,2022-06-28 08:47:52+00:00,j23cat,halo pls lewat tl dong hpku kecemplung di wc i...,jelek,halo tolong lewat timeline dong hpku kecemplun...,"[halo, tolong, lewat, timeline, dong, hpku, ke..."


# STOP REMOVAL

In [17]:
from nltk.corpus import stopwords

list_stopwords = nltk.corpus.stopwords.words('indonesian')
st = pd.read_csv('dataset/stopwords.csv')

list_stopwords.extend(st)
list_stopwords =set(list_stopwords)


def remove_stopwords(tweet):
    tweet = [word for word in tweet if word not in list_stopwords]
    return tweet
df['STOP_REMOVAL'] = df['TOKENIZATION'].apply(lambda x: remove_stopwords(x))
df.head(10)

,date,username,tweet,keyword,Normalisasi,TOKENIZATION,STOP_REMOVAL
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat,jelek,cel ketawanya jelek banget,"[cel, ketawanya, jelek, banget]","[cel, ketawanya, jelek, banget]"
1,2022-06-28 08:48:41+00:00,imasshana,mau req foto ktp pke filter ig deh muka gue ...,jelek,mau request foto ktp pakai filter ig deh muka ...,"[mau, request, foto, ktp, pakai, filter, ig, d...","[request, foto, ktp, pakai, filter, ig, deh, m..."
2,2022-06-28 08:48:35+00:00,junhuicy,nyoba pake kamera analog dan hasilnya jelek ...,jelek,nyoba pake kamera analog dan hasilnya jelek in...,"[nyoba, pake, kamera, analog, dan, hasilnya, j...","[nyoba, pake, kamera, analog, hasilnya, jelek,..."
3,2022-06-28 08:48:33+00:00,melicoww,jelek kali emotmu itu yh yank kmu juga teta...,jelek,jelek kali emotmu itu yah sayang kamu juga tet...,"[jelek, kali, emotmu, itu, yah, sayang, kamu, ...","[jelek, kali, emotmu, yah, sayang, menyerah, s..."
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek,sepertinya nickname aku kalau di tambah hala j...,"[sepertinya, nickname, aku, kalau, di, tambah,...","[nickname, hala, jelek, anjing]"
5,2022-06-28 08:48:18+00:00,renjunkw,kebetulan ara jelek,jelek,kebetulan ara jelek,"[kebetulan, ara, jelek]","[ara, jelek]"
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta seandainya bi...,jelek,beruntung banget cinta itu buta seandainya bis...,"[beruntung, banget, cinta, itu, buta, seandain...","[beruntung, banget, cinta, buta, seandainya, n..."
7,2022-06-28 08:47:59+00:00,nosyacey,emg udh jelek sih tpi itu jeleknya pake bange...,jelek,emang udah jelek sih tapi itu jeleknya pake ba...,"[emang, udah, jelek, sih, tapi, itu, jeleknya,...","[emang, udah, jelek, sih, jeleknya, pake, bang..."
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek,aku sayang banget sama provider tri meskipun s...,"[aku, sayang, banget, sama, provider, tri, mes...","[sayang, banget, provider, tri, sinyal, jelek,..."
9,2022-06-28 08:47:52+00:00,j23cat,halo pls lewat tl dong hpku kecemplung di wc i...,jelek,halo tolong lewat timeline dong hpku kecemplun...,"[halo, tolong, lewat, timeline, dong, hpku, ke...","[halo, tolong, timeline, hpku, kecemplung, wc,..."


# STEMMING

In [18]:
factory = StemmerFactory();
stemmer = factory.create_stemmer()

#stemed

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df ['STOP_REMOVAL']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
print(len(term_dict))
print('---------')
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
print(term_dict)

# apply
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['STEMMED'] = df['STOP_REMOVAL'].apply(lambda x: get_stemmed_term(x))
df.head(10)


3144
---------
{'cel': 'cel', 'ketawanya': 'ketawa', 'jelek': 'jelek', 'banget': 'banget', 'request': 'request', 'foto': 'foto', 'ktp': 'ktp', 'pakai': 'pakai', 'filter': 'filter', 'ig': 'ig', 'deh': 'deh', 'muka': 'muka', 'deket': 'deket', 'kamera': 'kamera', 'njir': 'njir', 'edan': 'edan', 'nyoba': 'nyoba', 'pake': 'pake', 'analog': 'analog', 'hasilnya': 'hasil', 'sih': 'sih', 'wkwkwk': 'wkwkwk', 'sebel': 'sebel', 'susah': 'susah', 'yak': 'yak', 'kali': 'kali', 'emotmu': 'emotmu', 'yah': 'yah', 'sayang': 'sayang', 'menyerah': 'serah', 'semangat': 'semangat', 'nickname': 'nickname', 'hala': 'hala', 'anjing': 'anjing', 'ara': 'ara', 'beruntung': 'untung', 'cinta': 'cinta', 'buta': 'buta', 'seandainya': 'anda', 'ngeliat': 'ngeliat', 'orang': 'orang', 'laku': 'laku', 'emang': 'emang', 'udah': 'udah', 'jeleknya': 'jelek', 'bangetnya': 'banget', 'provider': 'provider', 'tri': 'tri', 'sinyal': 'sinyal', 'suka': 'suka', 'trouble': 'trouble', 'hahahahhaha': 'hahahahhaha', 'marah': 'marah', 'h

,date,username,tweet,keyword,Normalisasi,TOKENIZATION,STOP_REMOVAL,STEMMED
0,2022-06-28 08:48:50+00:00,ieumaki,cel ketawanya jelek bat,jelek,cel ketawanya jelek banget,"[cel, ketawanya, jelek, banget]","[cel, ketawanya, jelek, banget]","[cel, ketawa, jelek, banget]"
1,2022-06-28 08:48:41+00:00,imasshana,mau req foto ktp pke filter ig deh muka gue ...,jelek,mau request foto ktp pakai filter ig deh muka ...,"[mau, request, foto, ktp, pakai, filter, ig, d...","[request, foto, ktp, pakai, filter, ig, deh, m...","[request, foto, ktp, pakai, filter, ig, deh, m..."
2,2022-06-28 08:48:35+00:00,junhuicy,nyoba pake kamera analog dan hasilnya jelek ...,jelek,nyoba pake kamera analog dan hasilnya jelek in...,"[nyoba, pake, kamera, analog, dan, hasilnya, j...","[nyoba, pake, kamera, analog, hasilnya, jelek,...","[nyoba, pake, kamera, analog, hasil, jelek, si..."
3,2022-06-28 08:48:33+00:00,melicoww,jelek kali emotmu itu yh yank kmu juga teta...,jelek,jelek kali emotmu itu yah sayang kamu juga tet...,"[jelek, kali, emotmu, itu, yah, sayang, kamu, ...","[jelek, kali, emotmu, yah, sayang, menyerah, s...","[jelek, kali, emotmu, yah, sayang, serah, sema..."
4,2022-06-28 08:48:31+00:00,cuddlyhwa,keknya nickname w klo di tambah hala jelek anjir,jelek,sepertinya nickname aku kalau di tambah hala j...,"[sepertinya, nickname, aku, kalau, di, tambah,...","[nickname, hala, jelek, anjing]","[nickname, hala, jelek, anjing]"
5,2022-06-28 08:48:18+00:00,renjunkw,kebetulan ara jelek,jelek,kebetulan ara jelek,"[kebetulan, ara, jelek]","[ara, jelek]","[ara, jelek]"
6,2022-06-28 08:48:05+00:00,C_eunw0_0,beruntung banget cinta itu buta seandainya bi...,jelek,beruntung banget cinta itu buta seandainya bis...,"[beruntung, banget, cinta, itu, buta, seandain...","[beruntung, banget, cinta, buta, seandainya, n...","[untung, banget, cinta, buta, anda, ngeliat, o..."
7,2022-06-28 08:47:59+00:00,nosyacey,emg udh jelek sih tpi itu jeleknya pake bange...,jelek,emang udah jelek sih tapi itu jeleknya pake ba...,"[emang, udah, jelek, sih, tapi, itu, jeleknya,...","[emang, udah, jelek, sih, jeleknya, pake, bang...","[emang, udah, jelek, sih, jelek, pake, banget,..."
8,2022-06-28 08:47:56+00:00,madathestar,aku sayang banget sama provider tri meskipun s...,jelek,aku sayang banget sama provider tri meskipun s...,"[aku, sayang, banget, sama, provider, tri, mes...","[sayang, banget, provider, tri, sinyal, jelek,...","[sayang, banget, provider, tri, sinyal, jelek,..."
9,2022-06-28 08:47:52+00:00,j23cat,halo pls lewat tl dong hpku kecemplung di wc i...,jelek,halo tolong lewat timeline dong hpku kecemplun...,"[halo, tolong, lewat, timeline, dong, hpku, ke...","[halo, tolong, timeline, hpku, kecemplung, wc,...","[halo, tolong, timeline, hpku, cemplung, wc, s..."


In [ ]:
df.to_csv('dataset/data_preprocessing.csv', index=False)
df.head(5)